In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import cadquery as cq
from cq_jupyter import Assembly, Part

In [ ]:
b = cq.Workplane('XY')
box1 = b.box(1,2,3).edges(">X").chamfer(0.1)
box2 = b.transformed(offset=cq.Vector(0, 1.5, 0.8)).box(3,2,1).edges(">Z").fillet(0.1)
box1.cut(box2)

Assembly(
    Part(box1, "red box", (1,0,0), visible=True), 
    Part(box2, "green box", (0,1,0), visible=True),
    height=600,   # height of the view div
    ortho=True,   # use orthographic view
    fov=0.4,      # field of view (used only for ortho=False)
    debug=False   # print html string including the x3dom string for debugging
)

# Phone holder 

In [ ]:
length = 70
width = 80
height = 8.5
thickness = 1.5
deviceRadius = 7
innerRadius = (height - 0.5) / 2.0
padding = 11.0 - deviceRadius

b = cq.Workplane('XY')
f = cq.Workplane('XZ')

inner = (
    b.box(length, width, height)
       .edges('|Z').edges('<Y').fillet(deviceRadius)
       .edges("|Y").edges("<Y").fillet(innerRadius)
)

outer = (
    b.box(length + thickness, width + 2 * thickness, height + thickness)
        .edges('|Z').edges('<Y').fillet(deviceRadius + thickness / 2)  
        .edges("|Y").edges("<Y").fillet(innerRadius + thickness/2)
)

front_cutter = (
    f.transformed(offset=cq.Vector(0, 0, -width/2))
    .box(length + 2 * thickness, height + 2 * thickness, 2 * thickness)
)

top_cutter = (
    b.transformed(offset=cq.Vector(0, thickness/2, (height)/2))
    .box(length - deviceRadius, width - deviceRadius + 2* thickness, 2 * thickness)
    .edges('|Z').edges('<Y').fillet(innerRadius)  
)

bottom_cutter = (
    b.transformed(offset=cq.Vector(0, -width/2 + thickness, thickness))
    .box(length - deviceRadius - padding, deviceRadius + 2 * thickness, height + thickness)
    .edges('|Y').edges('<Y').fillet(height/2 + thickness/4)
)

shell = (
    outer
    .cut(inner)
    .cut(front_cutter)
    .cut(top_cutter)
    .cut(bottom_cutter)
)

Assembly(
    Part(shell,         "shell",         (1,1,0)), 
    Part(front_cutter,  "front_cutter",  (1,0,0), visible=False),
    Part(top_cutter,    "top_cutter",    (0,1,0), visible=False),
    Part(bottom_cutter, "bottom_cutter", (0,0,1), visible=False),
    height=600,
    ortho=True,
    fov=0.1,      # used only for ortho=False 
    debug=False
)

In [ ]:
from cadquery import exporters

In [ ]:
with open("hh.stl", "w") as fd:
    exporters.exportShape(shell, "STL", fd)